this is for ipyhton only

In [ ]:
%matplotlib inline

Lets first import some stuff

In [ ]:
# These are imported by Sputter funcs already but it doesn't harm to import tham again
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import log as ln
from itertools import cycle # used to create a loopable colormap

This defines the location of a file that holds all the atomic data we need. the files holds values fro the calculation of the sputter yield according to Sigmund as weel as Matsunami1984 and Yamamura1996. The latter two modify the calculation in a way that makes it more suitable for litgh ions aka He and Ne.

In [ ]:
SRIM_ATOM_DATA_FILE="ATOMDATA.csv"
SRIM_He_FILE="He-30kV-SRIM-sputter-yield.dat"
SRIM_Ne_FILE="Ne-30kV-SRIM-sputter-yield.dat"
NA=6.022e23

Lets define what we want to calculate
==========

Projectile
----------

In [ ]:
Projectile="He"

Target
------

In [ ]:
Target="Si"

Energy
------

In [ ]:
Energy=25000

Surface binding energy
-----
Use this to fix the surface displacment energy to a fixed value, which is not in the database. If this is set to -1 the tabulated value will be used in all other cases User_SBW will be used instead of the tabulated value.

In [ ]:
User_SBE=-1

Which Sputter model should we use
----
can be either _Sigmund_, _M84_ or _Y96_

In [ ]:
sputter_models=["Sigmund","M84","Y96"]
sputter="Y96"

Reduced nuclear stopping cross section
-----
We support two model namely Thomas Fermi (_TF_) or Krypton Carbon (_KrC_)

In [ ]:
model="TF"

Lets read in the data

In [ ]:
'''
Try to read in the datafile as pandas frame
'''
Atom_props=pd.read_table(SRIM_ATOM_DATA_FILE,delim_whitespace=True)
#Atom_props.get_value('He','Natural Mass')

More data to read from SRIm simulations so for only for He 30kV

In [ ]:
Y_He_srim=pd.read_csv(SRIM_He_FILE,index_col='Ordnungszahl')
Y_He_srim['y30_He_srim']=Y_He_srim[' Anzahl Gesputterte']/Y_He_srim[' Anzahl Simulierte']

Y_He_srim contains the results of a SRIM simulation of the sputter yield for 30 keV He into the elements. Y_Ne_srim the same data for Ne.

In [ ]:
Y_Ne_srim=pd.read_csv(SRIM_Ne_FILE,header=None)
Y_Ne_srim['y30_Ne_srim']=Y_Ne_srim[1]/Y_Ne_srim[2]

In [ ]:
# Lets merge the Ne data from SRIM with the ATOMDATA
Atom_props=pd.merge(Atom_props,Y_Ne_srim.drop([1,2],axis=1),left_on='Z',right_on=0)
# Next we merge the He data from SRIM and drop the additional column "0" thet is left from merging
Atom_props=pd.merge(Atom_props,Y_He_srim.drop([' Anzahl Gesputterte',' Anzahl Simulierte'],axis=1),left_on='Z',right_index=True).drop(0,axis=1)

All the different values will be computed in functions so we can call them at anytime, and reuse them for making graphs and similar stuff. These functions are defined in a separate file so they can be reused in other projects.

In [ ]:
%run 'Sputter yield functions.ipynb'

All the results
===

In [ ]:
print ('for {:s} into {:s} using an energy of {:d} eV.'.format(Projectile, Target, Energy))
print ('Y(Sigmund)={:2g}'.format(yE(Projectile,Target,Energy,"Sigmund",model)))
print ('Y(Matsunami)={:2g}'.format(yE(Projectile,Target,Energy,"M84",model)))
print ('Y(Yamamura)={:2g}'.format(yE(Projectile,Target,Energy,"Y96",model)))
print ('For reference we quote the result of the SRIM simulation done at 30 keV')
print ('Y(He,30keV)={:2g}'.format(get_Atom_prop(Target,'y30_He_srim')))
print ('Y(Ne,30keV)={:2g}'.format(get_Atom_prop(Target,'y30_Ne_srim')))

Intermediate results leading to the above numbers:

In [ ]:
print ('eps={:3g}'.format(reduced_energy(Projectile,Target,Energy,sputter)))
print ('sn=%.3g'%reduced_nuc_stopping_x_section(Projectile,Target,Energy,sputter,model))
print ('se=%.3g'%(reduced_elec_stopping_x_section(Projectile,Target,Energy,sputter)))
print ('K=%.3g'%(Sn(Projectile,Target,Energy,sputter,model)/reduced_nuc_stopping_x_section(Projectile,Target,Energy,sputter,model)))
print ('Sn=%.3g'%(Sn(Projectile,Target,Energy,sputter,model)))
print ('a*=%.3g'%(alpha_stern(Projectile,Target,sputter)))
if User_SBE==-1:
    print ('Us=%.3g'%(get_Atom_prop(Target,'SBE')))
else:
    print ('Us=%.3g'%(User_SBE))
print ('Eth=%.3g'%(eth(Projectile,Target,sputter)))

The following variable holds the ions we use for the energy dependent plot of the target material (ziel).

In [ ]:
Ion=["He","Ne","Ga"]

First we create a plot that shows the sputter yeild for the selected ion (proj) for all possible targets.

Create the xaxis i.e. fill it with the atoms where all data ia available

In [ ]:
'''
Lets calculate the sputter yields for all elements
'''


for sputter in sputter_models:
    Atom_props["yE_"+str(sputter)]=None
    for i in Atom_props.index:
        if (sputter=="Y96")&(Atom_props.ix[i,'SBE']!=0)&(Atom_props.ix[i,'Q96']!=0):
            # If we use M84 we have more elements available
            # we only calculate for the ones where we do have SBE
            Atom_props.ix[i, "yE_"+str(sputter)] =yE(str(Projectile),Atom_props.Symbol.iloc[i],Energy,sputter,model)
        elif Atom_props.ix[i,'SBE']!=0:
            Atom_props.ix[i, "yE_"+str(sputter)] =yE(str(Projectile),Atom_props.Symbol.iloc[i],Energy,sputter,model) 

Lets make a figure that plots the sputter yeild for all elements when proj is used. Several energies are analyzied although the difference is small.

In [ ]:
cycol = cycle('bgrcmk') # this uses the itertools module to create a loopable colormap

# Create the figure and the axes object so we loop over all models
plt.figure()
ax=plt.axes()

for sputter in sputter_models:
    Atom_props.plot.scatter(x='Z',y="yE_"+str(sputter),color=next(cycol),label=sputter,ax=ax)

# Let's also add the srim data
Atom_props.plot.line(x='Z',y='y30_He_srim',color=next(cycol),label='30keV He srim',ax=ax)
Atom_props.plot.line(x='Z',y='y30_Ne_srim',color=next(cycol),label='30keV Ne srim',ax=ax)

# format the plot
ax.set_yscale("log", nonposy='clip')
plt.axis('tight')
plt.legend(loc='upper left')
plt.title(str(Energy/1000)+" keV "+Projectile)
plt.xlabel('Atomimc number [Z]')
plt.ylabel('Sputter yield [atoms/ion]')
plt.show(ax)
#filename="YE_"+Projectile+"_"+str(Energy)+".pdf"
#plt.savefig(filename)



Make a new figure for the energy dependent plots

In [ ]:
# Specify the starting and end energies
E_min=5000
E_max=35000
# and create an array of logspaced values in this range
Energien=np.logspace(np.log10(E_min),np.log10(E_max),10)

In [ ]:
plt.figure(2)
plt.clf()
plt.cla()
plt.hold('on')
# Lets go and iterate over the various ions and energies
j=0
for c in range(len(Ion)):
    y=[]
    for j in range(len(Energien)):
        y.append(yE(Ion[c],Target,Energien[j],sputter,model))
    plt.loglog(Energien,y,'-',label=Ion[c],nonposy='mask')
#Finaly beautify the graph
plt.legend(loc='lower right')
text="Sputteryield of "+Target+" using "+sputter+" and "+model
plt.title(text)
plt.xlabel('Energy [eV]')
plt.ylabel('Sputter yield [atoms/ion]')
plt.axis('tight')
plt.show
filename="YE_"+Target+"_"+sputter+"_"+model+".pdf"
plt.savefig(filename)